## Install

In [ ]:
!pip install langchain_community langchain chromadb rouge-score

In [ ]:
!pip install -U langchain-huggingface

In [ ]:
!pip install langchain-groq

In [ ]:
import numpy as np
import shutil
import zipfile
import os
import torch
import difflib
import chromadb
import json
import pandas as pd
import matplotlib.pyplot as plt
import re
from langchain_community.vectorstores import Chroma
from langchain.schema import Document
from langchain.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline, AutoModelForCausalLM, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.schema.retriever import BaseRetriever
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
import torch.nn.functional as F

from tqdm import tqdm
from operator import itemgetter
from langchain.chains import LLMChain
from rouge_score import rouge_scorer
from pprint import pprint
from collections import Counter
from huggingface_hub import hf_hub_download
from typing import List
from tqdm import tqdm
from json import dumps, loads
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from sentence_transformers import CrossEncoder
from huggingface_hub import HfApi, HfFolder, upload_file
from huggingface_hub import login
from langchain_groq import ChatGroq
from google.colab import files

## Test data(Use private dataset)

#### Data Preprocessing

In [ ]:
# === Read data ===
with open("private_dataset.json", "r", encoding="utf-8") as f:
    data = json.load(f)
df = pd.DataFrame(data)

# === Split/Chunk ===
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=120,
    length_function=len,
    separators=[".", " "]
)

In [ ]:
section_counter = Counter()

for text in df["full_text"]:
    sections = re.findall(r"(^|\n)([A-Z][a-zA-Z \-/]{2,100})\n", text)
    for _, sec in sections:
        section_counter[sec.strip()] += 1

# === Section ===
for sec, count in section_counter.most_common():
    print(f"{sec}: {count}")

In [ ]:
# === Data Preprocesssing ===
def preprocess_text(text):
    return re.sub(r'https?://\S+', lambda m: f"[URL]", text)

remove_sections = {
    "References",
    "Acknowledgments",
    "Acknowledgements",
    "Acknowledgment",
    "Acknowledgement",
    "Appendix",
}

def remove_sections_from_text(text, remove_sections):
    pattern = re.compile(r"\n{3}([A-Z][a-zA-Z \-/]{2,100})\n")
    matches = list(pattern.finditer(text))
    remove_set = {s.lower() for s in remove_sections}
    spans_to_remove = []
    removed_sections = []

    for i, match in enumerate(matches):
        section_title = match.group(1).strip()
        if section_title.lower() in remove_set:
            start = match.start()
            end = matches[i + 1].start() if i + 1 < len(matches) else len(text)
            removed_text = text[start:end]
            removed_sections.append((section_title, removed_text))
            spans_to_remove.append((start, end))

    for start, end in reversed(spans_to_remove):
        text = text[:start] + text[end:]

    return text, removed_sections

def clean_chunk(chunk):
    cleaned = chunk.lstrip(". ").strip()


    if cleaned:
        last_sentence = re.split(r'[.?!]\s+', cleaned)[-1]
        if not re.search(r'[.?!]$', last_sentence):
            cleaned += "."

    return cleaned

def chunk_with_section_titles(full_text, title=""):
    full_text = preprocess_text(full_text)
    if not full_text.strip():
        return []

    sections = full_text.split("\n\n\n")
    remove_set = {s.lower() for s in remove_sections}
    all_chunks = []

    for sec_index, section in enumerate(sections):
        lines = section.strip().split("\n", 1)

        # 嘗試拆出 section 標題與內容
        if len(lines) == 2:
            section_title = lines[0].strip()
            content = lines[1].strip()

            # 如果是要移除的 section，跳過
            if section_title.lower() in remove_set:
                print("=" * 60)
                print(f"🔴 刪除了 section: {section_title}")
                print(section.strip())
                print("=" * 60)
                continue
        else:
            content = lines[0].strip()

        # 對內容進行 chunk 切分（不包含 section 標題）
        chunks = text_splitter.split_text(content)
        cleaned_chunks = [clean_chunk(c) for c in chunks if clean_chunk(c)]

        all_chunks.extend(cleaned_chunks)

    return all_chunks

#### Add Data in Vector Store

In [ ]:
documents_section = []

for i, row in df.iterrows():
    title = row.get("title", "")
    full_text = row.get("full_text", "")

    # 第二種 主題/段落 chunking
    for j, chunk in enumerate(chunk_with_section_titles(full_text, title)):
        documents_section.append(Document(
            page_content=chunk,
            metadata={
                "chunk_id": f"{i}_{j}",
                "title": title
            }
        ))

In [ ]:
# === Chromadb ===
chroma_client = chromadb.PersistentClient(path="/content/chroma_dbtest2")

# === Embedding ===
embedding_function = SentenceTransformerEmbeddingFunction(
    model_name="BAAI/bge-base-en-v1.5"
)

try:
    chroma_client.delete_collection("my_collection")
except:
    pass

collection = chroma_client.get_or_create_collection(
    name="my_collection",
    embedding_function=embedding_function
)

# === Add data in Chromadb ===
def add_documents_to_chroma(docs, prefix="doc"):
    batch_size = 100
    for i in range(0, len(docs), batch_size):
        batch = docs[i:i + batch_size]
        texts = [doc.page_content for doc in batch]
        metadatas = [doc.metadata for doc in batch]
        ids = [f"{prefix}_{i + j}" for j in range(len(batch))]

        for m in metadatas:
            for k, v in m.items():
                if isinstance(v, (list, dict)):
                    m[k] = json.dumps(v)

        collection.add(
            documents=texts,
            metadatas=metadatas,
            ids=ids
        )


add_documents_to_chroma(documents_section)

print("✅ 已成功將所有 chunk 加入 ChromaDB！")

✅ 已成功將所有 chunk 加入 ChromaDB！


In [ ]:
shutil.make_archive("/content/chroma_dbtest2", 'zip', "/content/chroma_dbtest2")

'/content/chroma_dbtest2.zip'

In [ ]:
# === Use Llama for Question Generation, Temp Answer Generation ===
from transformers import AutoModelForCausalLM
from huggingface_hub import login

login("hf_xxxxxx")  # 換成你的 API Token
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"  # 或改成你訓練過的模型

tokenizer = AutoTokenizer.from_pretrained(model_id)
model_judge = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype="auto")

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model_judge,
    tokenizer=tokenizer,
    max_new_tokens=128,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=[tokenizer.eos_token_id,tokenizer.convert_tokens_to_ids("<|eot_id|>")],
    do_sample=False
)
llm_answer = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda:0


In [ ]:
# === Read data from Chromadb ===
embedding = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")
vector_store = Chroma(
    collection_name="my_collection",
    embedding_function=embedding,
    persist_directory="/content/chroma_dbtest2"
)

In [ ]:
# ===== My answer LLM ======
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    api_key="gsk_aaaaaaa",
    temperature=0.0,
    max_tokens=128,
    model_kwargs={
        "frequency_penalty": 0.8,
        "presence_penalty": 0.4,
    }
)

In [ ]:
# ====== Prompt =====
def build_single_input_prompt(title: str) -> ChatPromptTemplate:
    """
    title 為主題，context 為檢索出來的相關段落，question 為 QA 問題。
    """
    system_text = (
        "You are a helpful assistant. Use the retrieved context to answer the question accurately.\n\n"
        "Answer a question about the following topic:\n"
        f"=== Topic ===\n{title.strip()}\n\n"
        "Use the topic to identify the most relevant information in the retrieved content."
    )

    user_text = (
        "Instructions:\n"
        "- Carefully read the context and question.\n"
        "- Only use the retrieved context below to answer.\n"
        "- Do not add any outside information.\n"

        "=== Retrieved Context ===\n{{context}}\n\n"
        "Question: {{question}}\n"
        "Respond in this format:\n"
        "Answer: <brief answer>\n"
        "Think step by step before answering.\n\n"

    )

    prompt = ChatPromptTemplate.from_messages(
        [("system", system_text), ("user", user_text)],
        template_format="jinja2"
    )

    assert set(prompt.input_variables) == {"context", "question"}
    return prompt

In [ ]:
def generate_queries_from_llama(question: str) -> list[str]:
    messages = [
        {"role": "system", "content": (
            "You are a helpful assistant that expands a given research question into 1 additional diverse search queries.\n"
            "These queries should cover different perspectives, subtopics, or alternative phrasings of the original question.\n"
            "Return exactly 1 queries, one per line, without any explanations or numbering."
        )},
        {"role": "user", "content": f"Original Question: {question}\nGenerate 1 related but diverse search queries:"}
    ]

    input_ids = tokenizer.apply_chat_template(
        messages, return_tensors="pt", add_generation_prompt=True
    ).to(model_judge.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    output_ids = model_judge.generate(
        input_ids=input_ids,
        max_new_tokens=64,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=terminators,
        do_sample=False,
    )

    output_text = tokenizer.decode(output_ids[0][input_ids.shape[-1]:], skip_special_tokens=True)

    expanded_queries = [question.strip()]
    expanded_queries += [q.strip("-•0123456789. ") for q in output_text.splitlines() if q.strip()]
    return expanded_queries


In [ ]:
# === RRF ===
def reciprocal_rank_fusion(results: list[list[Document]], k=60):
    fused_scores = {}
    for docs in results:
        for rank, doc in enumerate(docs):
            key = (doc.page_content, json.dumps(doc.metadata, sort_keys=True))  # 使用元組作為唯一識別
            fused_scores[key] = fused_scores.get(key, 0) + 1 / (rank + k)

    reranked = sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    return [
        Document(page_content=content, metadata=json.loads(meta))
        for (content, meta), _ in reranked
    ]

In [ ]:
def retrieve_with_title_fallback(vector_store, title, question, top_k, verbose=False):

    # 先列出所有可用 title
    all_titles = list({
        m.get("title") for m in vector_store._collection.get(include=["metadatas"])["metadatas"]
        if "title" in m
    })

    # 嘗試找到最相近的 title（避免空格、dash、引號不一致）
    matched = difflib.get_close_matches(title, all_titles, n=1)
    used_title = matched[0] if matched else title
    # Step 1: title 相符的 documents
    results = vector_store._collection.query(
        query_texts=[question],
        n_results=top_k,
        where={"title": used_title}
    )


    docs = [
        Document(page_content=text, metadata=meta)
        for text, meta in zip(results["documents"][0], results["metadatas"][0])
    ]

    # Step 2: 若不足 top_k，補其他主題
    if len(docs) < top_k:
        needed = top_k - len(docs)
        fallback_results = vector_store._collection.query(
            query_texts=[question],
            n_results=needed,
            where={"title": {"$ne": used_title}}
        )
        fallback_docs = [
            Document(page_content=text, metadata=meta)
            for text, meta in zip(fallback_results["documents"][0], fallback_results["metadatas"][0])
        ]
        docs.extend(fallback_docs)

    if verbose:
        print(f"\nTop {top_k} Retrieved Chunks for title: '{title}' (matched: '{used_title}')")
        for idx, doc in enumerate(docs):
            print(f"\n--- Retrieved Chunk #{idx+1} ---")
            print(f"Title: {doc.metadata.get('title', '[No title]')}")
            print(f"Chunk ID: {doc.metadata.get('chunk_id')}")
            print(f"Content:\n{doc.page_content[:500]}...")

    return docs[:top_k]

class StaticRetriever(BaseRetriever):
    docs: List[Document]
    def _get_relevant_documents(self, query: str) -> List[Document]:
        return self.docs

In [ ]:
reranker = CrossEncoder("BAAI/bge-reranker-large", device="cuda")
def rerank_with_answer_based_bge(answer: str, docs: list[Document], top_k: int = 5, threshold: float = 0.6, verbose=False):
    pairs = [[answer, doc.page_content] for doc in docs]
    scores = reranker.predict(pairs)
    reranked = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)

    if verbose:
        print("\n📌 Rerank by Answer Similarity:")
        for i, (doc, score) in enumerate(reranked[:top_k]):
            print(f"# {i+1} | Score: {score:.4f} | Preview: {doc.page_content[:200]}...")

    return [doc for doc, score in reranked[:top_k] if score >= threshold] or [doc for doc, _ in reranked[:top_k]]


# === Retrieval based on RAG Fusion ===
def retrieve_rag_fusion(vector_store, title: str, question: str, per_query_k: int = 10, final_k: int = 20):
    queries = generate_queries_from_llama(question)

    all_results = []

    for idx, query in enumerate(queries):
        docs = retrieve_with_title_fallback(vector_store, title, query, top_k=per_query_k)
        all_results.append(docs)
        print(f"  - Query {idx+1}: {query} => {len(docs)} docs")  # debug info

    fused_docs = reciprocal_rank_fusion(all_results)

    print("\n🔁 RRF Top Documents:")
    for i, doc in enumerate(fused_docs[:final_k]):
        print(f"  🔁 RRF #{i+1} | Title: {doc.metadata.get('title')} | Preview: {doc.page_content[:200]}...")

    return fused_docs[:final_k]

In [ ]:
PER_QUERY_K = 10
FINAL_K_RRF = 15
FINAL_K_RERANK = 5
answers = []
results = []


checkpoint_path = "ragquerybgeanswerstd0.610_15_5test8b.json"

# === checkpoint）===
if os.path.exists(checkpoint_path):
    with open(checkpoint_path, "r", encoding="utf-8") as f:
        results = json.load(f)
    done_indices = {r["title"] for r in results}
else:
    results = []
    done_indices = set()

# === 主迴圈：遍歷驗證集 ===
for i, row in tqdm(df.iterrows(), total=len(df)):
    title = row["title"].strip()
    if title in done_indices:
        continue

    full_text = row["full_text"].strip()
    question = row["question"].strip()
    prompt = build_single_input_prompt(title)

    docs_before_rerank = retrieve_rag_fusion(
        vector_store, title, question,
        per_query_k=PER_QUERY_K, final_k=FINAL_K_RRF
    )

    # Step 1: Predict Temp Answer
    temp_retriever = StaticRetriever(docs=docs_before_rerank)
    temp_chain = RetrievalQA.from_chain_type(
        llm=llm_answer,
        retriever=temp_retriever,
        chain_type="stuff",
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=True
    )
    temp_result = temp_chain.invoke({"query": question})
    output_text = temp_result["result"]

    predicted_answer = ""
    for line in output_text.splitlines():
        if line.lower().startswith("answer:"):
            predicted_answer = line[len("answer:"):].strip()

    # Step 2: Rerank by Temp Answer
    fused_docs = rerank_with_answer_based_bge(predicted_answer, docs_before_rerank, top_k=FINAL_K_RERANK)
    final_retriever = StaticRetriever(docs=fused_docs)

    # Step 3: Answer
    rag_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=final_retriever,
        chain_type="stuff",
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=True
    )
    result = rag_chain.invoke({"query": question})

    answer_text = ""
    for line in result["result"].splitlines():
        if line.lower().startswith("answer:"):
            answer_text = line[len("answer:"):].strip()

    retrieved_chunks = [doc.page_content for doc in fused_docs]

    item = {
        "title": title,
        "answer": answer_text,
        "evidence": retrieved_chunks

    }
    results.append(item)

    with open(checkpoint_path, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)


## Train data(Public Dataset)




In [ ]:
# === Read Data ===
with open("public_dataset.json", "r", encoding="utf-8") as f:
    data = json.load(f)
df = pd.DataFrame(data)

# === Chunk/Split ===
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=120,
    length_function=len,
    separators=[".", " "]
)

In [ ]:
# === Section ===
section_counter = Counter()

for text in df["full_text"]:
    sections = re.findall(r"(^|\n)([A-Z][a-zA-Z \-/]{2,100})\n", text)
    for _, sec in sections:
        section_counter[sec.strip()] += 1

for sec, count in section_counter.most_common():
    print(f"{sec}: {count}")

In [ ]:
def preprocess_text(text):
    return re.sub(r'https?://\S+', lambda m: f"[URL]", text)
remove_sections = {
    "References", "Acknowledgments", "Appendix","Acknowledgements"
}

def remove_sections_from_text(text, remove_sections):
    pattern = re.compile(r"\n{3}([A-Z][a-zA-Z \-/]{2,100})\n")
    matches = list(pattern.finditer(text))
    remove_set = {s.lower() for s in remove_sections}
    spans_to_remove = []
    removed_sections = []

    for i, match in enumerate(matches):
        section_title = match.group(1).strip()
        if section_title.lower() in remove_set:
            start = match.start()
            end = matches[i + 1].start() if i + 1 < len(matches) else len(text)
            removed_text = text[start:end]
            removed_sections.append((section_title, removed_text))
            spans_to_remove.append((start, end))

    for start, end in reversed(spans_to_remove):
        text = text[:start] + text[end:]

    return text, removed_sections
def clean_chunk(chunk):
    cleaned = chunk.lstrip(". ").strip()

    if cleaned:
        last_sentence = re.split(r'[.?!]\s+', cleaned)[-1]
        if not re.search(r'[.?!]$', last_sentence):
            cleaned += "."

    return cleaned

def chunk_with_section_titles(full_text, title=""):
    full_text = preprocess_text(full_text)
    if not full_text.strip():
        return []

    sections = full_text.split("\n\n\n")
    remove_set = {s.lower() for s in remove_sections}
    all_chunks = []

    for sec_index, section in enumerate(sections):
        lines = section.strip().split("\n", 1)

        if len(lines) == 2:
            section_title = lines[0].strip()
            content = lines[1].strip()


            if section_title.lower() in remove_set:
                print("=" * 60)
                print(f"🔴 刪除了 section: {section_title}")
                print(section.strip())
                print("=" * 60)
                continue
        else:
            content = lines[0].strip()

        chunks = text_splitter.split_text(content)
        cleaned_chunks = [clean_chunk(c) for c in chunks if clean_chunk(c)]

        all_chunks.extend(cleaned_chunks)

    return all_chunks


#### Add Data in Vector Store

In [ ]:
documents_section = []

for i, row in df.iterrows():
    title = row.get("title", "")
    full_text = row.get("full_text", "")

    for j, chunk in enumerate(chunk_with_section_titles(full_text, title)):
        documents_section.append(Document(
            page_content=chunk,
            metadata={
                "chunk_id": f"{i}_{j}",
                "title": title
            }
        ))

In [ ]:
# === ChromaDB===
chroma_client = chromadb.PersistentClient(path="/content/chroma_db10")

# === embedding function===
embedding_function = SentenceTransformerEmbeddingFunction(
    model_name="BAAI/bge-base-en-v1.5"
)

try:
    chroma_client.delete_collection("my_collection")
except:
    pass

collection = chroma_client.get_or_create_collection(
    name="my_collection",
    embedding_function=embedding_function
)

# === Add data in ChromaDB ===
def add_documents_to_chroma(docs, prefix="doc"):
    batch_size = 100
    for i in range(0, len(docs), batch_size):
        batch = docs[i:i + batch_size]
        texts = [doc.page_content for doc in batch]
        metadatas = [doc.metadata for doc in batch]
        ids = [f"{prefix}_{i + j}" for j in range(len(batch))]

        for m in metadatas:
            for k, v in m.items():
                if isinstance(v, (list, dict)):
                    m[k] = json.dumps(v)

        collection.add(
            documents=texts,
            metadatas=metadatas,
            ids=ids
        )


add_documents_to_chroma(documents_section)


print("✅ 已成功將所有 chunk 加入 ChromaDB！")

In [ ]:
shutil.make_archive("/content/chroma_db10", 'zip', "/content/chroma_db10")

'/content/chroma_db10.zip'

In [ ]:
# === Step 2: 建立 LLM（使用 huggingface pipeline 包裝）===
from transformers import AutoModelForCausalLM
from huggingface_hub import login

login("hf_xxxxx")  # 換成你的 API Token
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"  # 或改成你訓練過的模型

tokenizer = AutoTokenizer.from_pretrained(model_id)
model_judge = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype="auto")

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model_judge,
    tokenizer=tokenizer,
    max_new_tokens=100,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=[tokenizer.eos_token_id,tokenizer.convert_tokens_to_ids("<|eot_id|>")],
    do_sample=False
)
llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda:0


In [ ]:
# === Read ChromaDB ===
embedding = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")
vector_store = Chroma(
    collection_name="my_collection",
    embedding_function=embedding,
    persist_directory="/content/chroma_db10"
)

#### Retrival Method

In [ ]:
with open("public_dataset.json", "r", encoding="utf-8") as f:
    data = json.load(f)
df = pd.DataFrame(data)

In [ ]:
##================Prompt======================
def build_single_input_prompt(title: str) -> ChatPromptTemplate:
    """
    title 為主題，context 為檢索出來的相關段落，question 為 QA 問題。
    """
    system_text = (
        "You are a helpful assistant. Use the retrieved context to answer the question accurately.\n\n"
        "Answer a question about the following topic:\n"
        f"=== Topic ===\n{title.strip()}\n\n"
        "Use the topic to identify the most relevant information in the retrieved content."
    )

    user_text = (
        "Instructions:\n"
        "- Carefully read the context and question.\n"
        "- Only use the retrieved context below to answer.\n"
        "- Do not add any outside information.\n"

        "=== Retrieved Context ===\n{{context}}\n\n"
        "Question: {{question}}\n"
        "Respond in this format:\n"
        "Answer: <brief answer>\n"
        "Think step by step before answering.\n\n"

    )

    prompt = ChatPromptTemplate.from_messages(
        [("system", system_text), ("user", user_text)],
        template_format="jinja2"
    )

    assert set(prompt.input_variables) == {"context", "question"}
    return prompt

In [ ]:
##===============Retrieval==============
def retrieve_with_title_fallback(vector_store, title, question, top_k, verbose=False):

    all_titles = list({
        m.get("title") for m in vector_store._collection.get(include=["metadatas"])["metadatas"]
        if "title" in m
    })

    # 嘗試找到最相近的 title（避免空格、dash、引號不一致）
    matched = difflib.get_close_matches(title, all_titles, n=1)
    used_title = matched[0] if matched else title
    # Step 1: title 相符的 documents
    results = vector_store._collection.query(
        query_texts=[question],
        n_results=top_k,
        where={"title": used_title}
    )

    docs = [
        Document(page_content=text, metadata=meta)
        for text, meta in zip(results["documents"][0], results["metadatas"][0])
    ]

    # Step 2: 若不足 top_k，補其他主題
    if len(docs) < top_k:
        needed = top_k - len(docs)
        fallback_results = vector_store._collection.query(
            query_texts=[question],
            n_results=needed,
            where={"title": {"$ne": used_title}}
        )
        fallback_docs = [
            Document(page_content=text, metadata=meta)
            for text, meta in zip(fallback_results["documents"][0], fallback_results["metadatas"][0])
        ]
        docs.extend(fallback_docs)

    if verbose:
        print(f"\nTop {top_k} Retrieved Chunks for title: '{title}' (matched: '{used_title}')")
        for idx, doc in enumerate(docs):
            print(f"\n--- Retrieved Chunk #{idx+1} ---")
            print(f"Title: {doc.metadata.get('title', '[No title]')}")
            print(f"Chunk ID: {doc.metadata.get('chunk_id')}")
            print(f"Content:\n{doc.page_content[:500]}...")

    return docs[:top_k]

class StaticRetriever(BaseRetriever):
    docs: List[Document]
    def _get_relevant_documents(self, query: str) -> List[Document]:
        return self.docs

##### 1. Step1

###### **1.** Query→Retrieve Top K

In [ ]:
TOP_K = 8
answers = []
results = []


checkpoint_path = "rag500_8.json"

if os.path.exists(checkpoint_path):
    with open(checkpoint_path, "r", encoding="utf-8") as f:
        results = json.load(f)
    done_indices = {r["title"] for r in results}  # 用 title 作為索引（視情況也可加 question）
else:
    results = []
    done_indices = set()

for i, row in tqdm(df.iterrows(), total=len(df)):
    title = row["title"].strip()
    if title in done_indices:
        continue  # 跳過已處理過的

    full_text = row["full_text"].strip()
    question = row["question"].strip()
    ground_truth_answer = " ".join(row["answer"]).strip()
    ground_truth_evidence = " ".join(row["evidence"]).strip()

    prompt = build_single_input_prompt(title)

    retrieved_docs = retrieve_with_title_fallback(vector_store, title, question, top_k=TOP_K)
    retriever = StaticRetriever(docs=retrieved_docs)

    rag_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type="stuff",
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=True
    )

    result = rag_chain.invoke({"query": question})
    output_text = result["result"]
    retrieved_chunks = [doc.page_content for doc in result["source_documents"]]
    evidence_text = " ".join(retrieved_chunks)


    answer_text = ""
    for line in output_text.splitlines():
        if line.lower().startswith("answer:"):
            answer_text = line[len("answer:"):].strip()


    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    rouge_result = scorer.score(ground_truth_evidence, evidence_text)
    rouge_l_score = rouge_result["rougeL"].fmeasure

    item = {
        "title": title,
        "question": question,
        "predicted_answer": answer_text,
        "ground_truth_answer": ground_truth_answer,
        "predicted_evidence": retrieved_chunks,
        "ground_truth_evidence": ground_truth_evidence,
        "rougeL": round(rouge_l_score, 4),
    }
    results.append(item)

    # === ✅ 寫入 checkpoint（每筆一存）===
    with open(checkpoint_path, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

    print(f"✅ Checkpoint saved for: {title}")

for idx, res in enumerate(results[:3]):
    print(f"--- Sample {idx+1} ---")
    print(f"Title: {res['title']}")
    print(f"Question: {res['question']}")
    print(f"Predicted Answer: {res['predicted_answer']}")
    print(f"Ground Truth Answer: {res['ground_truth_answer']}")
    print(f"Predicted Evidence: {res['predicted_evidence']}")
    print(f"Ground Truth Evidence: {res['ground_truth_evidence']}")
    print(f"ROUGE-L: {res['rougeL']}")
    print("-" * 80)


In [ ]:
import pandas as pd

# === 將結果轉成 DataFrame ===
results_df = pd.DataFrame(results)

rouge_scores = [res["rougeL"] for res in results]

avg_rouge = np.mean(rouge_scores)
lot_evidence = sum(1 for s in rouge_scores if s >= 0.7)
half_evidence = sum(1 for s in rouge_scores if s >= 0.5)
enough_evidence = sum(1 for s in rouge_scores if s >= 0.3)

print("\n=== 📊 驗證集整體評估結果 ===")
print(f"📈 平均 ROUGE-L 分數：{avg_rouge:.4f}")
print(f"📈 超多證據：{lot_evidence:.4f}")
print(f"📈 一半證據：{half_evidence:.4f}")
print(f"📈 有提到證據：{enough_evidence:.4f}")
print(f"📈 平均 ROUGE-L 分數：{avg_rouge:.4f}")


=== 📊 驗證集整體評估結果 ===
📈 平均 ROUGE-L 分數：0.1749
📈 超多證據：0.0000
📈 一半證據：0.0000
📈 有提到證據：17.0000
📈 平均 ROUGE-L 分數：0.1749


##### 2. Step2

###### **2.** Query→Retrieve→Rerank

In [ ]:
bge_tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-reranker-large")
bge_model = AutoModelForSequenceClassification.from_pretrained("BAAI/bge-reranker-large")

In [ ]:
bge_model.eval()
bge_model.cuda()
def rerank_with_bge(question: str, docs: list, top_k: int = 8):
    pairs = [(question, doc.page_content) for doc in docs]

    encoded = bge_tokenizer.batch_encode_plus(
        pairs,
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=512
    ).to(bge_model.device)

    with torch.no_grad():
        scores = bge_model(**encoded).logits.squeeze(-1)


    sorted_results = sorted(zip(docs, scores.tolist()), key=lambda x: x[1], reverse=True)

    return [doc for doc, _ in sorted_results[:top_k]]

In [ ]:
TOP_K = 15
RERANK_K = 5

answers = []
results = []

checkpoint_path = "rag500_15_5.json"


if os.path.exists(checkpoint_path):
    with open(checkpoint_path, "r", encoding="utf-8") as f:
        results = json.load(f)
    done_indices = {r["title"] for r in results}  # 用 title 作為索引（視情況也可加 question）
else:
    results = []
    done_indices = set()

# === 主迴圈：遍歷驗證集 ===
for i, row in tqdm(df.iterrows(), total=len(df)):
    title = row["title"].strip()
    if title in done_indices:
        continue

    full_text = row["full_text"].strip()
    question = row["question"].strip()
    ground_truth_answer = " ".join(row["answer"]).strip()
    ground_truth_evidence = " ".join(row["evidence"]).strip()

    prompt = build_single_input_prompt(title)

    retrieved_docs = retrieve_with_title_fallback(vector_store, title, question, top_k=TOP_K)
    reranked_docs = rerank_with_bge(question, retrieved_docs, top_k=RERANK_K)
    retriever = StaticRetriever(docs=reranked_docs)


    rag_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type="stuff",
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=True
    )

    result = rag_chain.invoke({"query": question})
    output_text = result["result"]

    retrieved_chunks = [doc.page_content for doc in result["source_documents"]]
    evidence_text = " ".join(retrieved_chunks)

    answer_text = ""
    for line in output_text.splitlines():
        if line.lower().startswith("answer:"):
            answer_text = line[len("answer:"):].strip()


    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    rouge_result = scorer.score(ground_truth_evidence, evidence_text)
    rouge_l_score = rouge_result["rougeL"].fmeasure


    item = {
        "title": title,
        "question": question,
        "predicted_answer": answer_text,
        "ground_truth_answer": ground_truth_answer,
        "predicted_evidence": retrieved_chunks,
        "ground_truth_evidence": ground_truth_evidence,
        "rougeL": round(rouge_l_score, 4),
    }
    results.append(item)

    # === ✅ 寫入 checkpoint（每筆一存）===
    with open(checkpoint_path, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

    print(f"✅ Checkpoint saved for: {title}")

# === 印出前三筆結果 ===
for idx, res in enumerate(results[:3]):
    print(f"--- Sample {idx+1} ---")
    print(f"Title: {res['title']}")
    print(f"Question: {res['question']}")
    print(f"Predicted Answer: {res['predicted_answer']}")
    print(f"Ground Truth Answer: {res['ground_truth_answer']}")
    print(f"Predicted Evidence: {res['predicted_evidence']}")
    print(f"Ground Truth Evidence: {res['ground_truth_evidence']}")
    print(f"ROUGE-L: {res['rougeL']}")
    print("-" * 80)


In [ ]:
#### k=15/5
import pandas as pd

# === 將結果轉成 DataFrame ===
results_df = pd.DataFrame(results)


rouge_scores = [res["rougeL"] for res in results]

avg_rouge = np.mean(rouge_scores)
lot_evidence = sum(1 for s in rouge_scores if s >= 0.7)
half_evidence = sum(1 for s in rouge_scores if s >= 0.5)
enough_evidence = sum(1 for s in rouge_scores if s >= 0.3)

print("\n=== 📊 驗證集整體評估結果 ===")
print(f"📈 平均 ROUGE-L 分數：{avg_rouge:.4f}")
print(f"📈 超多證據：{lot_evidence:.4f}")
print(f"📈 一半證據：{half_evidence:.4f}")
print(f"📈 有提到證據：{enough_evidence:.4f}")
print(f"📈 平均 ROUGE-L 分數：{avg_rouge:.4f}")


=== 📊 驗證集整體評估結果 ===
📈 平均 ROUGE-L 分數：0.1880
📈 超多證據：0.0000
📈 一半證據：1.0000
📈 有提到證據：20.0000
📈 平均 ROUGE-L 分數：0.1880


##### 3. Step 3

In [ ]:
reranker = CrossEncoder("BAAI/bge-reranker-large", device="cuda")

In [ ]:
# === RRF ===
def reciprocal_rank_fusion(results: list[list[Document]], k=60):
    fused_scores = {}
    for docs in results:
        for rank, doc in enumerate(docs):
            key = (doc.page_content, json.dumps(doc.metadata, sort_keys=True))  # 使用元組作為唯一識別
            fused_scores[key] = fused_scores.get(key, 0) + 1 / (rank + k)

    reranked = sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    return [
        Document(page_content=content, metadata=json.loads(meta))
        for (content, meta), _ in reranked
    ]

In [ ]:
def rerank_with_bge(query: str, docs: list[Document], top_k: int = 8):
    pairs = [[query, doc.page_content] for doc in docs]
    scores = reranker.predict(pairs)
    reranked = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)

    return [doc for doc, _ in reranked[:top_k]]

###### **3.** Query Augmentation → RRF → Rerank

In [ ]:
def generate_queries_from_llama(question: str) -> list[str]:
    messages = [
        {"role": "system", "content": (
            "You are a helpful assistant that expands a given research question into 2 additional diverse search queries.\n"
            "These queries should cover different perspectives, subtopics, or alternative phrasings of the original question.\n"
            "Return exactly 2 queries, one per line, without any explanations or numbering."
        )},
        {"role": "user", "content": f"Original Question: {question}\nGenerate 2 related but diverse search queries:"}
    ]

    input_ids = tokenizer.apply_chat_template(
        messages, return_tensors="pt", add_generation_prompt=True
    ).to(model_judge.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    output_ids = model_judge.generate(
        input_ids=input_ids,
        max_new_tokens=64,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=terminators,
        do_sample=False,
    )

    output_text = tokenizer.decode(output_ids[0][input_ids.shape[-1]:], skip_special_tokens=True)

    # 回傳：原始 + 延伸三個 query
    expanded_queries = [question.strip()]  # 原始查詢第一筆
    expanded_queries += [q.strip("-•0123456789. ") for q in output_text.splitlines() if q.strip()]
    return expanded_queries


In [ ]:
# === Retrieval based on RAG Fusion ===
def retrieve_rag_fusion(vector_store, title: str, question: str, per_query_k: int = 10, final_k: int = 20):
    queries = generate_queries_from_llama(question)
    all_results = []

    for idx, query in enumerate(queries):
        docs = retrieve_with_title_fallback(vector_store, title, query, top_k=per_query_k)
        all_results.append(docs)
        #print(f"  - Query {idx+1}: {query} => {len(docs)} docs")  # debug info

    fused_docs = reciprocal_rank_fusion(all_results)

    print("\n🔁 RRF Top Documents:")
    for i, doc in enumerate(fused_docs[:final_k]):
        print(f"  🔁 RRF #{i+1} | Title: {doc.metadata.get('title')} | Preview: {doc.page_content[:200]}...")

    return fused_docs[:final_k]

In [ ]:
PER_QUERY_K = 10
FINAL_K_RRF = 15
FINAL_K_RERANK = 5
answers = []
results = []


checkpoint_path = "ragfusionbge10_15_5all1.json"

# === 載入已儲存的 checkpoint（若存在）===
if os.path.exists(checkpoint_path):
    with open(checkpoint_path, "r", encoding="utf-8") as f:
        results = json.load(f)
    done_indices = {r["title"] for r in results}  # 用 title 作為索引（視情況也可加 question）
else:
    results = []
    done_indices = set()


for i, row in tqdm(df.iterrows(), total=len(df)):
    title = row["title"].strip()
    if title in done_indices:
        continue  # 跳過已處理過的

    full_text = row["full_text"].strip()
    question = row["question"].strip()
    ground_truth_answer = " ".join(row["answer"]).strip()
    ground_truth_evidence = " ".join(row["evidence"]).strip()

    prompt = build_single_input_prompt(title)

    docs_before_rerank = retrieve_rag_fusion(vector_store, title, question, per_query_k=PER_QUERY_K, final_k=FINAL_K_RRF)
    fused_docs = rerank_with_bge(question, docs_before_rerank, top_k=FINAL_K_RERANK)
    retriever = StaticRetriever(docs=fused_docs)

    rag_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type="stuff",
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=True
    )

    result = rag_chain.invoke({"query": question})
    output_text = result["result"]

    retrieved_chunks = [doc.page_content for doc in fused_docs]
    evidence_text = " ".join(retrieved_chunks)  # 一個字串，供模型與 ROUGE 使用

    answer_text = ""
    for line in output_text.splitlines():
        if line.lower().startswith("answer:"):
            answer_text = line[len("answer:"):].strip()

    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    rouge_result = scorer.score(ground_truth_evidence, evidence_text)
    rouge_l_score = rouge_result["rougeL"].fmeasure

    item = {
        "title": title,
        "question": question,
        "predicted_answer": answer_text,
        "ground_truth_answer": ground_truth_answer,
        "predicted_evidence": retrieved_chunks,
        "ground_truth_evidence": ground_truth_evidence,
        "rougeL": round(rouge_l_score, 4),

    }
    results.append(item)

    with open(checkpoint_path, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)


for idx, res in enumerate(results[:3]):
    print(f"--- Sample {idx+1} ---")
    print(f"Title: {res['title']}")
    print(f"Question: {res['question']}")
    print(f"Predicted Answer: {res['predicted_answer']}")
    print(f"Ground Truth Answer: {res['ground_truth_answer']}")
    print(f"Predicted Evidence: {res['predicted_evidence']}")
    print(f"Ground Truth Evidence: {res['ground_truth_evidence']}")
    print(f"ROUGE-L: {res['rougeL']}")
    print("-" * 80)


In [ ]:
###all10/15/5
results_df = pd.DataFrame(results)


rouge_scores = [res["rougeL"] for res in results]

avg_rouge = np.mean(rouge_scores)
lot_evidence = sum(1 for s in rouge_scores if s >= 0.7)
half_evidence = sum(1 for s in rouge_scores if s >= 0.5)
enough_evidence = sum(1 for s in rouge_scores if s >= 0.3)

print("\n=== 📊 驗證集整體評估結果 ===")
print(f"📈 平均 ROUGE-L 分數：{avg_rouge:.4f}")
print(f"📈 超多證據：{lot_evidence:.4f}")
print(f"📈 一半證據：{half_evidence:.4f}")
print(f"📈 有提到證據：{enough_evidence:.4f}")
print(f"📈 平均 ROUGE-L 分數：{avg_rouge:.4f}")


=== 📊 驗證集整體評估結果 ===
📈 平均 ROUGE-L 分數：0.1857
📈 超多證據：0.0000
📈 一半證據：0.0000
📈 有提到證據：22.0000
📈 平均 ROUGE-L 分數：0.1857


###### **4.** Answer Hypothesis → RRF → Rerank

In [ ]:
def generate_hypothesis_queries(question: str) -> list[str]:
    messages = [
        {"role": "system", "content": (
            "You are a knowledgeable research assistant. "
            "Based on the research question, hypothesize 2 possible answers (concise statements). "
            "Only return the 2 hypotheses, one per line, no explanations."
        )},
        {"role": "user", "content": f"Question: {question}"}
    ]

    input_ids = tokenizer.apply_chat_template(
        messages, return_tensors="pt", add_generation_prompt=True
    ).to(model_judge.device)

    output_ids = model_judge.generate(
        input_ids=input_ids,
        max_new_tokens=80,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=[tokenizer.eos_token_id],
        do_sample=False
    )

    output_text = tokenizer.decode(output_ids[0][input_ids.shape[-1]:], skip_special_tokens=True)

    hypotheses = [question.strip()]
    hypotheses += [h.strip("-•0123456789. ") for h in output_text.splitlines() if h.strip()]
    return hypotheses


In [ ]:
def retrieve_rag_fusion(vector_store, title: str, question: str, per_query_k: int = 10, final_k: int = 20):
    queries = generate_hypothesis_queries(question)
    all_results = []

    for idx, query in enumerate(queries):
        docs = retrieve_with_title_fallback(vector_store, title, query, top_k=per_query_k)
        all_results.append(docs)
        print(f"  - Query {idx+1}: {query} => {len(docs)} docs")  # debug info

    fused_docs = reciprocal_rank_fusion(all_results)


    return fused_docs[:final_k]

In [ ]:
PER_QUERY_K = 10
FINAL_K_RRF = 15
FINAL_K_RERANK = 5
answers = []
results = []


checkpoint_path = "raghypobge10_15_5all.json"

if os.path.exists(checkpoint_path):
    with open(checkpoint_path, "r", encoding="utf-8") as f:
        results = json.load(f)
    done_indices = {r["title"] for r in results}  # 用 title 作為索引（視情況也可加 question）
else:
    results = []
    done_indices = set()

for i, row in tqdm(df.iterrows(), total=len(df)):
    title = row["title"].strip()
    if title in done_indices:
        continue  # 跳過已處理過的

    full_text = row["full_text"].strip()
    question = row["question"].strip()
    ground_truth_answer = " ".join(row["answer"]).strip()
    ground_truth_evidence = " ".join(row["evidence"]).strip()

    prompt = build_single_input_prompt(title)

    docs_before_rerank = retrieve_rag_fusion(vector_store, title, question, per_query_k=PER_QUERY_K, final_k=FINAL_K_RRF)
    fused_docs = rerank_with_bge(question, docs_before_rerank, top_k=FINAL_K_RERANK)
    retriever = StaticRetriever(docs=fused_docs)

    rag_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type="stuff",
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=True
    )

    result = rag_chain.invoke({"query": question})
    output_text = result["result"]

    retrieved_chunks = [doc.page_content for doc in fused_docs]
    evidence_text = " ".join(retrieved_chunks)  # 一個字串，供模型與 ROUGE 使用

    answer_text = ""
    for line in output_text.splitlines():
        if line.lower().startswith("answer:"):
            answer_text = line[len("answer:"):].strip()

    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    rouge_result = scorer.score(ground_truth_evidence, evidence_text)
    rouge_l_score = rouge_result["rougeL"].fmeasure

    item = {
        "title": title,
        "question": question,
        "predicted_answer": answer_text,
        "ground_truth_answer": ground_truth_answer,
        "predicted_evidence": retrieved_chunks,
        "ground_truth_evidence": ground_truth_evidence,
        "rougeL": round(rouge_l_score, 4),
    }
    results.append(item)

    with open(checkpoint_path, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)


# === 印出前三筆結果 ===
for idx, res in enumerate(results[:3]):
    print(f"--- Sample {idx+1} ---")
    print(f"Title: {res['title']}")
    print(f"Question: {res['question']}")
    print(f"Predicted Answer: {res['predicted_answer']}")
    print(f"Ground Truth Answer: {res['ground_truth_answer']}")
    print(f"Predicted Evidence: {res['predicted_evidence']}")
    print(f"Ground Truth Evidence: {res['ground_truth_evidence']}")
    print(f"ROUGE-L: {res['rougeL']}")
    print("-" * 80)


In [ ]:
#all10-15-5
results_df = pd.DataFrame(results)


rouge_scores = [res["rougeL"] for res in results]

avg_rouge = np.mean(rouge_scores)
lot_evidence = sum(1 for s in rouge_scores if s >= 0.7)
half_evidence = sum(1 for s in rouge_scores if s >= 0.5)
enough_evidence = sum(1 for s in rouge_scores if s >= 0.3)

print("\n=== 📊 驗證集整體評估結果 ===")
print(f"📈 平均 ROUGE-L 分數：{avg_rouge:.4f}")
print(f"📈 超多證據：{lot_evidence:.4f}")
print(f"📈 一半證據：{half_evidence:.4f}")
print(f"📈 有提到證據：{enough_evidence:.4f}")
print(f"📈 平均 ROUGE-L 分數：{avg_rouge:.4f}")


=== 📊 驗證集整體評估結果 ===
📈 平均 ROUGE-L 分數：0.1674
📈 超多證據：0.0000
📈 一半證據：0.0000
📈 有提到證據：16.0000
📈 平均 ROUGE-L 分數：0.1674


###### **5.** Query Augmentation + Answer Hypothesis → RRF → Rerank

In [ ]:
def generate_queries_from_llama(question: str) -> list[str]:
    messages = [
        {"role": "system", "content": (
            "You are a helpful assistant that expands a given research question into 2 additional diverse search queries.\n"
            "These queries should cover different perspectives, subtopics, or alternative phrasings of the original question.\n"
            "Return exactly 2 queries, one per line, without any explanations or numbering."
        )},
        {"role": "user", "content": f"Original Question: {question}\nGenerate 2 related but diverse search queries:"}
    ]

    input_ids = tokenizer.apply_chat_template(
        messages, return_tensors="pt", add_generation_prompt=True
    ).to(model_judge.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    output_ids = model_judge.generate(
        input_ids=input_ids,
        max_new_tokens=64,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=terminators,
        do_sample=False,
    )

    output_text = tokenizer.decode(output_ids[0][input_ids.shape[-1]:], skip_special_tokens=True)

    expanded_queries = [question.strip()]  # 原始查詢第一筆
    expanded_queries += [q.strip("-•0123456789. ") for q in output_text.splitlines() if q.strip()]
    return expanded_queries


In [ ]:
def generate_hypothesis_queries_noorig(question: str) -> list[str]:
    messages = [
        {"role": "system", "content": (
            "You are a knowledgeable research assistant. "
            "Based on the research question, hypothesize 1 possible answers (concise statements). "
            "Only return the 1 hypotheses, one per line, no explanations."
        )},
        {"role": "user", "content": f"Question: {question}"}
    ]

    input_ids = tokenizer.apply_chat_template(
        messages, return_tensors="pt", add_generation_prompt=True
    ).to(model_judge.device)

    output_ids = model_judge.generate(
        input_ids=input_ids,
        max_new_tokens=80,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=[tokenizer.eos_token_id],
        do_sample=False
    )

    output_text = tokenizer.decode(output_ids[0][input_ids.shape[-1]:], skip_special_tokens=True)

    hypotheses = [h.strip("-•0123456789. ") for h in output_text.splitlines() if h.strip()]
    return hypotheses


In [ ]:
# === Retrieval based on RAG Fusion ===
def retrieve_rag_fusion(vector_store, title: str, question: str, per_query_k: int = 10, final_k: int = 20):
    queries = generate_queries_from_llama(question) + generate_hypothesis_queries_noorig(question)

    all_results = []

    for idx, query in enumerate(queries):
        docs = retrieve_with_title_fallback(vector_store, title, query, top_k=per_query_k)
        all_results.append(docs)
        print(f"  - Query {idx+1}: {query} => {len(docs)} docs")  # debug info

    fused_docs = reciprocal_rank_fusion(all_results)

    print("\n🔁 RRF Top Documents:")
    for i, doc in enumerate(fused_docs[:final_k]):
        print(f"  🔁 RRF #{i+1} | Title: {doc.metadata.get('title')} | Preview: {doc.page_content[:200]}...")

    return fused_docs[:final_k]

In [ ]:
PER_QUERY_K = 10
FINAL_K_RRF = 15
FINAL_K_RERANK = 5
answers = []
results = []


checkpoint_path = "raghypoquerybge10_15_5all.json"

if os.path.exists(checkpoint_path):
    with open(checkpoint_path, "r", encoding="utf-8") as f:
        results = json.load(f)
    done_indices = {r["title"] for r in results}  # 用 title 作為索引（視情況也可加 question）
else:
    results = []
    done_indices = set()

for i, row in tqdm(df.iterrows(), total=len(df)):
    title = row["title"].strip()
    if title in done_indices:
        continue

    full_text = row["full_text"].strip()
    question = row["question"].strip()
    ground_truth_answer = " ".join(row["answer"]).strip()
    ground_truth_evidence = " ".join(row["evidence"]).strip()

    prompt = build_single_input_prompt(title)

    docs_before_rerank = retrieve_rag_fusion(vector_store, title, question, per_query_k=PER_QUERY_K, final_k=FINAL_K_RRF)
    fused_docs = rerank_with_bge(question, docs_before_rerank, top_k=FINAL_K_RERANK)
    retriever = StaticRetriever(docs=fused_docs)

    rag_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type="stuff",
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=True
    )

    result = rag_chain.invoke({"query": question})
    output_text = result["result"]

    retrieved_chunks = [doc.page_content for doc in fused_docs]
    evidence_text = " ".join(retrieved_chunks)

    answer_text = ""
    for line in output_text.splitlines():
        if line.lower().startswith("answer:"):
            answer_text = line[len("answer:"):].strip()

    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    rouge_result = scorer.score(ground_truth_evidence, evidence_text)
    rouge_l_score = rouge_result["rougeL"].fmeasure

    item = {
        "title": title,
        "question": question,
        "predicted_answer": answer_text,
        "ground_truth_answer": ground_truth_answer,
        "predicted_evidence": retrieved_chunks,
        "ground_truth_evidence": ground_truth_evidence,
        "rougeL": round(rouge_l_score, 4),
    }
    results.append(item)

    # === ✅ 寫入 checkpoint（每筆一存）===
    with open(checkpoint_path, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)



# === 印出前三筆結果 ===
for idx, res in enumerate(results[:3]):
    print(f"--- Sample {idx+1} ---")
    print(f"Title: {res['title']}")
    print(f"Question: {res['question']}")
    print(f"Predicted Answer: {res['predicted_answer']}")
    print(f"Ground Truth Answer: {res['ground_truth_answer']}")
    print(f"Predicted Evidence: {res['predicted_evidence']}")
    print(f"Ground Truth Evidence: {res['ground_truth_evidence']}")
    print(f"ROUGE-L: {res['rougeL']}")
    print("-" * 80)


In [ ]:
#10-15-5
results_df = pd.DataFrame(results)


rouge_scores = [res["rougeL"] for res in results]

avg_rouge = np.mean(rouge_scores)
lot_evidence = sum(1 for s in rouge_scores if s >= 0.7)
half_evidence = sum(1 for s in rouge_scores if s >= 0.5)
enough_evidence = sum(1 for s in rouge_scores if s >= 0.3)

print("\n=== 📊 驗證集整體評估結果 ===")
print(f"📈 平均 ROUGE-L 分數：{avg_rouge:.4f}")
print(f"📈 超多證據：{lot_evidence:.4f}")
print(f"📈 一半證據：{half_evidence:.4f}")
print(f"📈 有提到證據：{enough_evidence:.4f}")


=== 📊 驗證集整體評估結果 ===
📈 平均 ROUGE-L 分數：0.1831
📈 超多證據：0.0000
📈 一半證據：0.0000
📈 有提到證據：20.0000


##### 4. Step4

###### **6.**Query Augmentation → RRF → Rerank → Score Filter

In [ ]:
def generate_queries_from_llama(question: str) -> list[str]:
    messages = [
        {"role": "system", "content": (
            "You are a helpful assistant that expands a given research question into 2 additional diverse search queries.\n"
            "These queries should cover different perspectives, subtopics, or alternative phrasings of the original question.\n"
            "Return exactly 2 queries, one per line, without any explanations or numbering."
        )},
        {"role": "user", "content": f"Original Question: {question}\nGenerate 2 related but diverse search queries:"}
    ]

    input_ids = tokenizer.apply_chat_template(
        messages, return_tensors="pt", add_generation_prompt=True
    ).to(model_judge.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    output_ids = model_judge.generate(
        input_ids=input_ids,
        max_new_tokens=64,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=terminators,
        do_sample=False,
    )

    output_text = tokenizer.decode(output_ids[0][input_ids.shape[-1]:], skip_special_tokens=True)

    expanded_queries = [question.strip()]
    expanded_queries += [q.strip("-•0123456789. ") for q in output_text.splitlines() if q.strip()]
    return expanded_queries


In [ ]:
def rerank_with_bge_standard(query: str, docs: list[Document], top_k: int = 8, verbose=False):
    pairs = [[query, doc.page_content] for doc in docs]
    scores = reranker.predict(pairs)
    reranked = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)

    filtered = [(doc, score) for doc, score in reranked if score > 0.6]

    if verbose:
        print("\n🔁 Rerank Result:")
        for rank, (doc, score) in enumerate(filtered if filtered else reranked[:top_k]):
            print(f"\n# {rank+1} | Score: {score:.4f} | Title: {doc.metadata.get('title', '[No title]')}")
            print(f"Chunk ID: {doc.metadata.get('chunk_id', '[No ID]')}")
            print(f"Content Preview:\n{doc.page_content[:300]}...")

    return [doc for doc, _ in (filtered if filtered else reranked[:top_k])]

# === Retrieval based on RAG Fusion ===
def retrieve_rag_fusion(vector_store, title: str, question: str, per_query_k: int = 10, final_k: int = 20):
    queries = generate_queries_from_llama(question)
    all_results = []

    for idx, query in enumerate(queries):
        docs = retrieve_with_title_fallback(vector_store, title, query, top_k=per_query_k)
        all_results.append(docs)
        print(f"  - Query {idx+1}: {query} => {len(docs)} docs")  # debug info


    fused_docs = reciprocal_rank_fusion(all_results)


    print("\n🔁 RRF Top Documents:")
    for i, doc in enumerate(fused_docs[:final_k]):
        print(f"  🔁 RRF #{i+1} | Title: {doc.metadata.get('title')} | Preview: {doc.page_content[:200]}...")

    return fused_docs[:final_k]

In [ ]:
PER_QUERY_K = 10
FINAL_K_RRF = 15
FINAL_K_RERANK = 5
answers = []
results = []


checkpoint_path = "ragfusionbge0.610_15_5all.json"

if os.path.exists(checkpoint_path):
    with open(checkpoint_path, "r", encoding="utf-8") as f:
        results = json.load(f)
    done_indices = {r["title"] for r in results}  # 用 title 作為索引（視情況也可加 question）
else:
    results = []
    done_indices = set()

for i, row in tqdm(df.iterrows(), total=len(df)):
    title = row["title"].strip()
    if title in done_indices:
        continue  # 跳過已處理過的

    full_text = row["full_text"].strip()
    question = row["question"].strip()
    ground_truth_answer = " ".join(row["answer"]).strip()
    ground_truth_evidence = " ".join(row["evidence"]).strip()

    prompt = build_single_input_prompt(title)

    docs_before_rerank = retrieve_rag_fusion(vector_store, title, question, per_query_k=PER_QUERY_K, final_k=FINAL_K_RRF)
    fused_docs = rerank_with_bge_standard(question, docs_before_rerank, top_k=FINAL_K_RERANK)
    retriever = StaticRetriever(docs=fused_docs)

    rag_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type="stuff",
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=True
    )

    result = rag_chain.invoke({"query": question})
    output_text = result["result"]

    retrieved_chunks = [doc.page_content for doc in fused_docs]
    evidence_text = " ".join(retrieved_chunks)

    answer_text = ""
    for line in output_text.splitlines():
        if line.lower().startswith("answer:"):
            answer_text = line[len("answer:"):].strip()

    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    rouge_result = scorer.score(ground_truth_evidence, evidence_text)
    rouge_l_score = rouge_result["rougeL"].fmeasure

    item = {
        "title": title,
        "question": question,
        "predicted_answer": answer_text,
        "ground_truth_answer": ground_truth_answer,
        "predicted_evidence": retrieved_chunks,
        "ground_truth_evidence": ground_truth_evidence,
        "rougeL": round(rouge_l_score, 4),

    }
    results.append(item)

    # === ✅ 寫入 checkpoint（每筆一存）===
    with open(checkpoint_path, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)


# === 印出前三筆結果 ===
for idx, res in enumerate(results[:3]):
    print(f"--- Sample {idx+1} ---")
    print(f"Title: {res['title']}")
    print(f"Question: {res['question']}")
    print(f"Predicted Answer: {res['predicted_answer']}")
    print(f"Ground Truth Answer: {res['ground_truth_answer']}")
    print(f"Predicted Evidence: {res['predicted_evidence']}")
    print(f"Ground Truth Evidence: {res['ground_truth_evidence']}")
    print(f"ROUGE-L: {res['rougeL']}")
    print("-" * 80)

In [ ]:
####all10-15-5
results_df = pd.DataFrame(results)


rouge_scores = [res["rougeL"] for res in results]

avg_rouge = np.mean(rouge_scores)
lot_evidence = sum(1 for s in rouge_scores if s >= 0.7)
half_evidence = sum(1 for s in rouge_scores if s >= 0.5)
enough_evidence = sum(1 for s in rouge_scores if s >= 0.3)

print("\n=== 📊 驗證集整體評估結果 ===")
print(f"📈 平均 ROUGE-L 分數：{avg_rouge:.4f}")
print(f"📈 超多證據：{lot_evidence:.4f}")
print(f"📈 一半證據：{half_evidence:.4f}")
print(f"📈 有提到證據：{enough_evidence:.4f}")
print(f"📈 平均 ROUGE-L 分數：{avg_rouge:.4f}")


=== 📊 驗證集整體評估結果 ===
📈 平均 ROUGE-L 分數：0.2043
📈 超多證據：3.0000
📈 一半證據：5.0000
📈 有提到證據：23.0000
📈 平均 ROUGE-L 分數：0.2043


###### **7.** Query Augmentation+Answer → RRF → Rerank → Score Filter

In [ ]:
def generate_queries_from_llama(question: str) -> list[str]:
    messages = [
        {"role": "system", "content": (
            "You are a helpful assistant that expands a given research question into 2 additional diverse search queries.\n"
            "These queries should cover different perspectives, subtopics, or alternative phrasings of the original question.\n"
            "Return exactly 2 queries, one per line, without any explanations or numbering."
        )},
        {"role": "user", "content": f"Original Question: {question}\nGenerate 2 related but diverse search queries:"}
    ]

    input_ids = tokenizer.apply_chat_template(
        messages, return_tensors="pt", add_generation_prompt=True
    ).to(model_judge.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    output_ids = model_judge.generate(
        input_ids=input_ids,
        max_new_tokens=64,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=terminators,
        do_sample=False,
    )

    output_text = tokenizer.decode(output_ids[0][input_ids.shape[-1]:], skip_special_tokens=True)

    expanded_queries = [question.strip()]
    expanded_queries += [q.strip("-•0123456789. ") for q in output_text.splitlines() if q.strip()]
    return expanded_queries


In [ ]:
def generate_hypothesis_queries_noorig(question: str) -> list[str]:
    messages = [
        {"role": "system", "content": (
            "You are a knowledgeable research assistant. "
            "Based on the research question, hypothesize 1 possible answers (concise statements). "
            "Only return the 1 hypotheses, one per line, no explanations."
        )},
        {"role": "user", "content": f"Question: {question}"}
    ]

    input_ids = tokenizer.apply_chat_template(
        messages, return_tensors="pt", add_generation_prompt=True
    ).to(model_judge.device)

    output_ids = model_judge.generate(
        input_ids=input_ids,
        max_new_tokens=80,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=[tokenizer.eos_token_id],
        do_sample=False
    )

    output_text = tokenizer.decode(output_ids[0][input_ids.shape[-1]:], skip_special_tokens=True)

    hypotheses = [h.strip("-•0123456789. ") for h in output_text.splitlines() if h.strip()]
    return hypotheses


In [ ]:
def rerank_with_bge_standard(query: str, docs: list[Document], top_k: int = 8, verbose=False):
    pairs = [[query, doc.page_content] for doc in docs]
    scores = reranker.predict(pairs)
    reranked = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)

    filtered = [(doc, score) for doc, score in reranked if score > 0.6]

    if verbose:
        print("\n🔁 Rerank Result:")
        for rank, (doc, score) in enumerate(filtered if filtered else reranked[:top_k]):
            print(f"\n# {rank+1} | Score: {score:.4f} | Title: {doc.metadata.get('title', '[No title]')}")
            print(f"Chunk ID: {doc.metadata.get('chunk_id', '[No ID]')}")
            print(f"Content Preview:\n{doc.page_content[:300]}...")

    return [doc for doc, _ in (filtered if filtered else reranked[:top_k])]

# === Retrieval based on RAG Fusion ===
def retrieve_rag_fusion(vector_store, title: str, question: str, per_query_k: int = 10, final_k: int = 20):
    queries = generate_queries_from_llama(question) + generate_hypothesis_queries_noorig(question)

    all_results = []

    for idx, query in enumerate(queries):
        docs = retrieve_with_title_fallback(vector_store, title, query, top_k=per_query_k)
        all_results.append(docs)
        print(f"  - Query {idx+1}: {query} => {len(docs)} docs")  # debug info

    fused_docs = reciprocal_rank_fusion(all_results)

    print("\n🔁 RRF Top Documents:")
    for i, doc in enumerate(fused_docs[:final_k]):
        print(f"  🔁 RRF #{i+1} | Title: {doc.metadata.get('title')} | Preview: {doc.page_content[:200]}...")

    return fused_docs[:final_k]

In [ ]:
PER_QUERY_K = 10
FINAL_K_RRF = 15
FINAL_K_RERANK = 5
answers = []
results = []


checkpoint_path = "raghypoquerybgestd0.610_15_5all.json"

if os.path.exists(checkpoint_path):
    with open(checkpoint_path, "r", encoding="utf-8") as f:
        results = json.load(f)
    done_indices = {r["title"] for r in results}
else:
    results = []
    done_indices = set()

for i, row in tqdm(df.iterrows(), total=len(df)):
    title = row["title"].strip()
    if title in done_indices:
        continue

    full_text = row["full_text"].strip()
    question = row["question"].strip()
    ground_truth_answer = " ".join(row["answer"]).strip()
    ground_truth_evidence = " ".join(row["evidence"]).strip()

    prompt = build_single_input_prompt(title)

    docs_before_rerank = retrieve_rag_fusion(vector_store, title, question, per_query_k=PER_QUERY_K, final_k=FINAL_K_RRF)
    fused_docs = rerank_with_bge_standard(question, docs_before_rerank, top_k=FINAL_K_RERANK)
    retriever = StaticRetriever(docs=fused_docs)

    rag_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type="stuff",
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=True
    )

    result = rag_chain.invoke({"query": question})
    output_text = result["result"]

    retrieved_chunks = [doc.page_content for doc in fused_docs]
    evidence_text = " ".join(retrieved_chunks)

    answer_text = ""
    for line in output_text.splitlines():
        if line.lower().startswith("answer:"):
            answer_text = line[len("answer:"):].strip()

    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    rouge_result = scorer.score(ground_truth_evidence, evidence_text)
    rouge_l_score = rouge_result["rougeL"].fmeasure

    item = {
        "title": title,
        "question": question,
        "predicted_answer": answer_text,
        "ground_truth_answer": ground_truth_answer,
        "predicted_evidence": retrieved_chunks,
        "ground_truth_evidence": ground_truth_evidence,
        "rougeL": round(rouge_l_score, 4),
    }
    results.append(item)

    with open(checkpoint_path, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)


# === 印出前三筆結果 ===
for idx, res in enumerate(results[:3]):
    print(f"--- Sample {idx+1} ---")
    print(f"Title: {res['title']}")
    print(f"Question: {res['question']}")
    print(f"Predicted Answer: {res['predicted_answer']}")
    print(f"Ground Truth Answer: {res['ground_truth_answer']}")
    print(f"Predicted Evidence: {res['predicted_evidence']}")
    print(f"Ground Truth Evidence: {res['ground_truth_evidence']}")
    print(f"ROUGE-L: {res['rougeL']}")
    print("-" * 80)

In [ ]:
##########all 10-15-5
results_df = pd.DataFrame(results)


rouge_scores = [res["rougeL"] for res in results]

avg_rouge = np.mean(rouge_scores)
lot_evidence = sum(1 for s in rouge_scores if s >= 0.7)
half_evidence = sum(1 for s in rouge_scores if s >= 0.5)
enough_evidence = sum(1 for s in rouge_scores if s >= 0.3)

print("\n=== 📊 驗證集整體評估結果 ===")
print(f"📈 平均 ROUGE-L 分數：{avg_rouge:.4f}")
print(f"📈 超多證據：{lot_evidence:.4f}")
print(f"📈 一半證據：{half_evidence:.4f}")
print(f"📈 有提到證據：{enough_evidence:.4f}")


=== 📊 驗證集整體評估結果 ===
📈 平均 ROUGE-L 分數：0.2017
📈 超多證據：3.0000
📈 一半證據：4.0000
📈 有提到證據：22.0000


##### 5. Step 5

###### **8.**Query Expansion → RRF Fusion → Temp Answer → Answer-Guided Rerank → Score Filter

In [ ]:
def generate_queries_from_llama(question: str) -> list[str]:
    messages = [
        {"role": "system", "content": (
            "You are a helpful assistant that expands a given research question into 1 additional diverse search queries.\n"
            "These queries should cover different perspectives, subtopics, or alternative phrasings of the original question.\n"
            "Return exactly 1 queries, one per line, without any explanations or numbering."
        )},
        {"role": "user", "content": f"Original Question: {question}\nGenerate 1 related but diverse search queries:"}
    ]

    input_ids = tokenizer.apply_chat_template(
        messages, return_tensors="pt", add_generation_prompt=True
    ).to(model_judge.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    output_ids = model_judge.generate(
        input_ids=input_ids,
        max_new_tokens=64,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=terminators,
        do_sample=False,
    )

    output_text = tokenizer.decode(output_ids[0][input_ids.shape[-1]:], skip_special_tokens=True)

    expanded_queries = [question.strip()]
    expanded_queries += [q.strip("-•0123456789. ") for q in output_text.splitlines() if q.strip()]
    return expanded_queries


In [ ]:
def rerank_with_answer_based_bge(answer: str, docs: list[Document], top_k: int = 5, threshold: float = 0.6, verbose=True):
    pairs = [[answer, doc.page_content] for doc in docs]
    scores = reranker.predict(pairs)
    reranked = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)

    if verbose:
        print("\n📌 Rerank by Answer Similarity:")
        for i, (doc, score) in enumerate(reranked[:top_k]):
            print(f"# {i+1} | Score: {score:.4f} | Preview: {doc.page_content[:200]}...")

    return [doc for doc, score in reranked[:top_k] if score >= threshold] or [doc for doc, _ in reranked[:top_k]]


# === Retrieval based on RAG Fusion ===
def retrieve_rag_fusion(vector_store, title: str, question: str, per_query_k: int = 10, final_k: int = 20):
    queries = generate_queries_from_llama(question)

    all_results = []

    for idx, query in enumerate(queries):
        docs = retrieve_with_title_fallback(vector_store, title, query, top_k=per_query_k)
        all_results.append(docs)
        print(f"  - Query {idx+1}: {query} => {len(docs)} docs")  # debug info

    fused_docs = reciprocal_rank_fusion(all_results)

    print("\n🔁 RRF Top Documents:")
    for i, doc in enumerate(fused_docs[:final_k]):
        print(f"  🔁 RRF #{i+1} | Title: {doc.metadata.get('title')} | Preview: {doc.page_content[:200]}...")

    return fused_docs[:final_k]

In [ ]:
PER_QUERY_K = 10
FINAL_K_RRF = 15
FINAL_K_RERANK = 5
answers = []
results = []


checkpoint_path = "ragquerybgeanswerstd0.610_15_5all.json"

if os.path.exists(checkpoint_path):
    with open(checkpoint_path, "r", encoding="utf-8") as f:
        results = json.load(f)
    done_indices = {r["title"] for r in results}  # 用 title 作為索引（視情況也可加 question）
else:
    results = []
    done_indices = set()

for i, row in tqdm((df).iterrows(), total=len(df)):
    title = row["title"].strip()
    if title in done_indices:
        continue

    full_text = row["full_text"].strip()
    question = row["question"].strip()
    ground_truth_answer = " ".join(row["answer"]).strip()
    ground_truth_evidence = " ".join(row["evidence"]).strip()

    prompt = build_single_input_prompt(title)

    docs_before_rerank = retrieve_rag_fusion(
        vector_store, title, question,
        per_query_k=PER_QUERY_K, final_k=FINAL_K_RRF
    )

    temp_retriever = StaticRetriever(docs=docs_before_rerank)
    temp_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=temp_retriever,
        chain_type="stuff",
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=True
    )
    temp_result = temp_chain.invoke({"query": question})
    output_text = temp_result["result"]

    predicted_answer = ""
    for line in output_text.splitlines():
        if line.lower().startswith("answer:"):
            predicted_answer = line[len("answer:"):].strip()

    fused_docs = rerank_with_answer_based_bge(predicted_answer, docs_before_rerank, top_k=FINAL_K_RERANK)
    final_retriever = StaticRetriever(docs=fused_docs)

    rag_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=final_retriever,
        chain_type="stuff",
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=True
    )
    result = rag_chain.invoke({"query": question})
    answer_text = ""
    for line in result["result"].splitlines():
        if line.lower().startswith("answer:"):
            answer_text = line[len("answer:"):].strip()

    retrieved_chunks = [doc.page_content for doc in fused_docs]
    evidence_text = " ".join(retrieved_chunks)

    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    rouge_result = scorer.score(ground_truth_evidence, evidence_text)
    rouge_l_score = rouge_result["rougeL"].fmeasure

    item = {
        "title": title,
        "question": question,
        "predicted_answer": answer_text,
        "ground_truth_answer": ground_truth_answer,
        "predicted_evidence": retrieved_chunks,
        "ground_truth_evidence": ground_truth_evidence,
        "rougeL": round(rouge_l_score, 4),
    }
    results.append(item)

    with open(checkpoint_path, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)


for idx, res in enumerate(results[:3]):
    print(f"--- Sample {idx+1} ---")
    print(f"Title: {res['title']}")
    print(f"Question: {res['question']}")
    print(f"Predicted Answer: {res['predicted_answer']}")
    print(f"Ground Truth Answer: {res['ground_truth_answer']}")
    print(f"Predicted Evidence: {res['predicted_evidence']}")
    print(f"Ground Truth Evidence: {res['ground_truth_evidence']}")
    print(f"ROUGE-L: {res['rougeL']}")
    print("-" * 80)

In [ ]:
#all10-15-5
results_df = pd.DataFrame(results)


rouge_scores = [res["rougeL"] for res in results]

avg_rouge = np.mean(rouge_scores)
lot_evidence = sum(1 for s in rouge_scores if s >= 0.7)
half_evidence = sum(1 for s in rouge_scores if s >= 0.5)
enough_evidence = sum(1 for s in rouge_scores if s >= 0.3)

print("\n=== 📊 驗證集整體評估結果 ===")
print(f"📈 平均 ROUGE-L 分數：{avg_rouge:.4f}")
print(f"📈 超多證據：{lot_evidence:.4f}")
print(f"📈 一半證據：{half_evidence:.4f}")
print(f"📈 有提到證據：{enough_evidence:.4f}")


=== 📊 驗證集整體評估結果 ===
📈 平均 ROUGE-L 分數：0.2809
📈 超多證據：7.0000
📈 一半證據：18.0000
📈 有提到證據：33.0000
